<a href="https://colab.research.google.com/github/desaiankitb/pytorch-basics/blob/main/tutorial-pytorch-org/00_4_Automatic_Differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic Differentiation

## Automatic Differentiation with `torch.autograd`

- When training neural networks, the most frequently used algorithm is **back prapagation**. In this algorithm, parameters (model weights) are adjusted according to the **gradient** of the loss function with respect to the given parameter. 

- To compute those gradients, PyTorch has a built-in diffrentiation engine called `torch.autograd`. It supports automatic computation of gradient for any computational graph. 

- Consider the simplest one-layer neural network, which input `x`, parameters `w` and `b`, and some loss function. It can be defined in PyTorch in the following manner: 

In [5]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output 
w = torch.randn(5, 3, requires_grad=True) 
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensors, Functions and Computational graph

This code defines the following **computational graph**:

![Diagram showing a computational graph with two parameters 'w' and 'b' to compute the gradients of loss.](https://drive.google.com/uc?export=view&id=1_hwyhjaJhQah0YS44R7jfrkHSYEyk_Mu)

- Note: [reference to use image in colab from drive](https://stackoverflow.com/questions/50670920/how-to-insert-an-inline-image-in-google-colaboratory-from-google-drive)

- In this network, `w` and `b` are **parameters**, which we need to optimize. Thus, we need to be able to compute the gradients of loss function with respect to those variables. In order to do that, we set the `required_grad` property of those tensors. 

> **Note:** You can set the value of `requires_grad` when creating a tensor, or later by using `x.requires_grad_(True)` method.

- A function that we apply to tensors to construct computational graph is in fact an object of class `Function`. This object knows how to compute the function in the *forward direction*, and also how to compute its derivative during the *backward prapagation* step. A reference to the backward prapogation function is stored in `grad_fn` property of a tensor. You can find more information of `Function` [in the documentation](https://pytorch.org/docs/stable/autograd.html#function).

In [6]:
print("Gradient function for z =", z.grad_fn)
print("Gradient function for loss = ", loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f34494b93d0>
Gradient function for loss =  <BinaryCrossEntropyWithLogitsBackward object at 0x7f34494b94d0>


## Computing gradients
- To optimize weights of parameters in the neural network, we need to compute the derivatives of our loss function with respect to parameters, namely, we need $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ under some fixed values of `x` and `y`. To compute those derivatives, we call `loss.backward()`, and then retrive the values from `w.grad` and `b.grad`:

In [7]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1361, 0.0341, 0.2563],
        [0.1361, 0.0341, 0.2563],
        [0.1361, 0.0341, 0.2563],
        [0.1361, 0.0341, 0.2563],
        [0.1361, 0.0341, 0.2563]])
tensor([0.1361, 0.0341, 0.2563])


> **Note**: We can only obtain the `grad` properties for the leaf node of the computational graph, which have `required_grad` property set to `True`. For all other nodes in our graph, gradients will not be available. In addition, we can only perform gradient calculations using `backward`  once on a given graph, for performance reasons. If we need to do serveral `backward` calls on the same graph, we need to pass, `retain_graph=True` to the backward call. 

## Disaling gradient tracking

- By default, all tensors with `requires_grad=True` are tracking their computational history and support gradient computation. However, there are some cases when we do not need to do that, for example, when we have trained the model and just want to apply it to some input data, i.e. we only want to do *forward* computations through the network. We can stop tracking computations by surrounding our computation code with `torch.no_grad()` block: 

In [8]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
  z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


- Another way to achieve the same result is to use `detach()` method on tensor: 

In [9]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


- There are reasons you might want to disable gradient tracking: 

  - To mark some parameters in your neural network at **frozen parameters**. This is very common scenario for [fine tuning a pre-trained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html).
  - To **speed up computations** when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient. 